<a href="https://colab.research.google.com/github/202318004/Assignments/blob/main/RecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from nltk.stem.porter import PorterStemmer
import re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


(5000, 6)

# Data Preprocessing

In [ ]:
class Data_Preprocessing():

  def __init__(self):
    self.ps = PorterStemmer()
    self.train_raw_path = "/content/drive/MyDrive/train.csv"
    self.train_processed_path = "/content/test.pkl"

  def remove_symbols(self,text):
    try:
      # Regular expression to match alphanumeric characters and spaces
      clean_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
      return clean_text
    except Exception as e:
      # Catch-all block for any other exceptions
      print(text)
      print(type(text))
      print("An unexpected error occurred:", e)

  def stem (self,text) :

    #print(text)
    #pattern=r'\b\d+\b'
    #text_without_numbers=re.sub(pattern,"",text)
    words = word_tokenize(text)
    stemmed_words=[self.ps.stem(word) for word in words]
    return ' '.join(stemmed_words)

  def read_pickle(self):
    # To read pickle file
    product=[]

    with open("/content/test.pkl","rb") as file:
      while True:
        try :
          product.append(pickle.load(file))
        except EOFError:
          break

    return pd.concat(product,ignore_index=True)

  def data_processing(self):

    with open(self.train_processed_path,"wb")as f:
      i=0
      for chunk in pd.read_csv("/content/drive/MyDrive/train.csv",chunksize=50000):
        i+=1
        if i==3 :
          break
        print(i)
        chunk = chunk[['PRODUCT_ID', 'TITLE', 'BULLET_POINTS', 'DESCRIPTION']]
        chunk = chunk[~chunk["TITLE"].isin(["",None,"nan"])]
        chunk = chunk[chunk["TITLE"].apply(lambda x: type(x) != float)]

        chunk["TITLE"]=chunk["TITLE"].apply(self.remove_symbols)

        chunk["TITLE"]=chunk["TITLE"].apply(self.stem)

        chunk = chunk[~chunk["DESCRIPTION"].isin(["",None,"nan"])]
        chunk = chunk[chunk["DESCRIPTION"].apply(lambda x: type(x) != float)]

        chunk["DESCRIPTION"]=chunk["DESCRIPTION"].apply(self.remove_symbols)

        chunk["DESCRIPTION"]=chunk["DESCRIPTION"].apply(self.stem)


        chunk = chunk[~chunk["BULLET_POINTS"].isin(["",None,"NaN"])]
        chunk = chunk[chunk["BULLET_POINTS"].apply(lambda x: type(x) != float)]

        chunk["BULLET_POINTS"]=chunk["BULLET_POINTS"].apply(self.remove_symbols)

        chunk["BULLET_POINTS"]=chunk["BULLET_POINTS"].apply(self.stem)
        print(chunk.shape)
        pickle.dump(chunk,f)


In [ ]:
dp = Data_Preprocessing()
dp.data_processing()
df=dp.read_pickle()


1
(22968, 4)
2
(23078, 4)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import gc

In [ ]:

product=[]

with open("/content/test.pkl","rb") as file:
  while True:
    try :
      product.append(pickle.load(file))
    except EOFError:
      break

product = pd.concat(product,ignore_index=True)


In [ ]:
tf = TfidfVectorizer(stop_words='english',max_features=5000)
tf.fit(product["TITLE"]+' '+product["DESCRIPTION"]+' '+product["BULLET_POINTS"])

TfidfVectorizer(max_features=5000, stop_words='english')

In [ ]:
for i in tf.get_feature_names_out() :
  print(i)

01
02
03
03mm
04
05
06
07
075
08
080
10
100
1000
101
1012
102
104
105
106
108
1080p
10cm
10i
10mm
10pc
10x
11
110
112
114
115
116
118
11d
11oz
12
120
1200
1214
1218
122
125
12cm
12inch
12mm
12v
12x12
12x18
13
130
135
138
13cm
13mm
14
140
144
145
14k
15
150
1500
155
156
157
15mm
16
160
165
16mm
16x16
17
175
177
18
180
185
188
18mm
18x18
19
190t
1deliveri
1in
1mm
1pc
1piec
1st
1x
1year
20
200
2000
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
20cm
20mm
21
210
22
220
225
22mm
23
230
235
236
24
240
245
25
250
2500
255
25d
25mm
26
27
275
276
28
29
295
2in1
2inch
2mm
2nd
2pack
2pc
2x
2xl
30
300
304
30cm
30day
31
315
316
32
325
32gb
33
330
335
34
34th
35
350
350ml
354
35mm
36
360
360degre
37
375
38
38mm
39
394
3d
3drose
3in1
3m
3mm
3nh
3pc
3rd
3x
3xl
40
400
40cm
40mm
41
42
425
42mm
43
44
44mm
45
450
45cm
46
47
48
480
49
4door
4g
4inch
4k
4mm
4pc
4th
4way
4x
4x4
4x6
4xl
50
500
5000
5050
50cm
51
510
512
52
53
54
55
550
56
57
58
59
5cm
5d
5g
5mm
5pc
5s

In [ ]:
first_iteration=True

print("Item Representation processing")
with open("/content/test.pkl",'rb') as f:
  with open("/content/test_item_rep.pkl", "wb") as file:
    i=0
    while True:
      try:
        i+=1

        chunk=pickle.load(f)
        #tfidf_matrix=self.tfidf_vectorizer.transform(chunk['description'] + ' ' + chunk['title']+' '+chunk['authors']+' '+chunk['publisher']).toarray()
        tfidf_matrix=pd.DataFrame(tf.transform(chunk['TITLE'] + ' ' + chunk['DESCRIPTION']+' '+chunk['BULLET_POINTS']).toarray())
        other_features = chunk[['PRODUCT_ID']]
        tfidf_matrix.reset_index(drop=True, inplace=True)
        other_features.reset_index(drop=True, inplace=True)
        joined_df = pd.concat([other_features,tfidf_matrix], axis=1)

        pickle.dump(joined_df, file)

        del chunk
        del tfidf_matrix
        del other_features
        del joined_df
        gc.collect()  # Perform garbage collection to free up memory

      except EOFError:
        print("Item Representation Task Completed")
        break

print("Item Representation Processed and stored")

Item Representation processing
Item Representation Task Completed
Item Representation Processed and stored


In [ ]:
product=[]

with open("/content/test_item_rep.pkl","rb") as file:
  while True:
    try :
      product.append(pickle.load(file))
    except EOFError:
      break

product = pd.concat(product,ignore_index=True)

print(product)

       PRODUCT_ID    0    1    2    3    4    5    6    7    8  ...  4990  \
0         2765088  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   
1         1594019  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   
2         2152929  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   
3         2026580  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   
4         2998633  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   
...           ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
46041     2285194  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   
46042     1620752  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   
46043     2964700  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   
46044     2013412  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   
46045     2082714  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   

       4991  4992  4993  4994  4995  4996  4997  4998  4999  
0       0.0  

In [ ]:
text = input("Enter Item_name :")
text=pd.DataFrame({"text":[text]})
print(text)
text_rep=tf.transform(text["text"]).toarray()
text_rep
products_id=product["PRODUCT_ID"]
#print(products_id.head())
similarity = cosine_similarity(product.drop(columns=["PRODUCT_ID"]),text_rep)
sim_df=pd.DataFrame(similarity)#.sort_values(by=0,ascending=False)
sim_df=pd.concat([products_id,sim_df],axis=1)
A=sim_df.sort_values(by=0,ascending=False).head(10)
A

Enter Item_name :yamaha
     text
0  yamaha


,PRODUCT_ID,0
38615,2286059,0.837922
45420,940698,0.753015
7971,803628,0.639436
41539,1443951,0.576217
38141,2239395,0.560160
6673,1138353,0.557464
44633,1194958,0.535840
5413,874608,0.512426
34513,1869054,0.463246
12841,1562680,0.430071


In [ ]:
def fetch_item_details(product_df):
    product_details=[]
    with open("/content/test.pkl",'rb') as f2:
      while True:
        try:
          condition = lambda x: x['PRODUCT_ID'].isin(product_df["PRODUCT_ID"])
          product_details.append(pickle.load(f2)[condition])



        except EOFError:
          break

    return pd.concat(product_details,ignore_index=True)

In [ ]:
fetch_item_details(A)

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION
0,874608,yana shiki soloy401b black metal x paint solo ...,fit yamaha yzfr1 20072008made with high qualit...,fit for yamaha yzfr1 brbrthese oem replac solo...
1,1138353,yamaha fc3a continu piano style sustain pedal,keyboard pedal that work with all halfdamp ena...,yamaha fc3a piano style sustain foot pedal wit...
2,803628,yamaha yac1295 5c standard bari saxophon mouth...,model yac1295can also be use for tenor saxtip ...,5c bariton saxophon mouthpiec the yamaha yac12...
3,1562680,spedi handl grip throttl cabl kill switch for ...,easi to instal set of 2 both side made from hi...,specif materi rubber plastic
4,1869054,open throttl racer yamaha r3 r25 mt03 header s...,high qualiti steelsustain to high degre of hea...,yamaha r3 header silver made from high qualiti...
5,2239395,big shopp store fuel tank valv petcock for yam...,100 brand newth bolt hole of center to center ...,pbdescriptionbpp100 brand newppth bolt hole of...
6,2286059,big shopp store 27 piec of set leather bariton...,it is design for yamaha bariton saxophone27pie...,pbdescriptionbppit is design for yamaha barito...
7,1443951,yamaha sax genuin saxophon maroon key guard fe...,yamaha genuin saxophon felt key guard bumperma...,yamaha genuin saxophon felt key guard bumperbr...
8,1194958,factori effex youth hood sweatshirt black xlar...,yamaha youth hoodi black xlthe yamaha youth pu...,offici licens hoodi
9,940698,oem yamaha 6070hp 2stroke outboard thermostat ...,oem yamaha 6070hp 2stroke outboard thermostat ...,oem yamaha outboard thermostat fit 60 c60 e60 ...
